# Model Class

In [1]:
class CarbonFootprintPredictor:
    def __init__(self):
        self.params = {
            'carbon_pijar': 0.02150,
            'carbon_neon': 0.00540,
            'carbon_led': 0.00240,
            'carbon_ac_non_inverter': 0.10396,
            'carbon_kulkas_inverter': 0.06500,
            'carbon_kulkas_non_inverter': 0.06600
        }

    def electric(self, kulkas_inverter, kulkas_num, kulkas_consume_hour, kulkas_power, ac_inverter, ac_num, ac_consume_hour, ac_power, lamp_type, lamp_num, lamp_consume_hour, lamp_power, others_power_total):
        power = (kulkas_num * kulkas_consume_hour * kulkas_power) + (ac_num * ac_consume_hour * ac_power) + (lamp_num * lamp_consume_hour * lamp_power) + others_power_total
        return power / 1000

    def calculate_carbon(self, kulkas_inverter, kulkas_num, kulkas_consume_hour, kulkas_power, ac_inverter, ac_num, ac_consume_hour, ac_power, lamp_type, lamp_num, lamp_consume_hour, lamp_power,others_power_total):
        electric_carbon = self.electric(kulkas_inverter, kulkas_num, kulkas_consume_hour, kulkas_power, ac_inverter, ac_num, ac_consume_hour, ac_power, lamp_type, lamp_num, lamp_consume_hour, lamp_power,others_power_total) * 0.0094

        lamp_carbon = self.params[f'carbon_{lamp_type}'] * lamp_consume_hour * lamp_num

        ac_carbon = ac_num * ac_consume_hour
        if ac_inverter == "belum":
            ac_carbon *= self.params['carbon_ac_non_inverter']

        kulkas_carbon = (kulkas_num / 24) * kulkas_consume_hour
        if kulkas_inverter == "sudah":
            kulkas_carbon *= self.params['carbon_kulkas_inverter']
        else:
            kulkas_carbon *= self.params['carbon_kulkas_non_inverter']

        carbon = electric_carbon + lamp_carbon + ac_carbon + kulkas_carbon
        carbon *= 1000
        return carbon / 12
    
    
    def predict(self, data):
        return self.calculate_carbon(**data)

    def kwh(self, data):
        return self.electric(**data)

In [2]:
import joblib

import joblib
model = CarbonFootprintPredictor()
joblib.dump(model, "carbon.joblib")

model = CarbonFootprintPredictor()
data = {
    "kulkas_inverter": "belum",
    "kulkas_num": 19,
    "kulkas_consume_hour": 24,
    "kulkas_power": 1,
    "ac_inverter": "belum",
    "ac_num": 20,
    "ac_consume_hour": 1,
    "ac_power":1,
    "lamp_type": "neon",
    "lamp_num": 39,
    "lamp_consume_hour": 10,
    "lamp_power": 1,
    "others_power_total": 100
}

In [3]:
carbon_footprint = model.predict(data)
print(f"Estimated monthly carbon footprint: {carbon_footprint:.2f} kg CO2 equivalent")

Estimated monthly carbon footprint: 454.02 kg CO2 equivalent


In [4]:
carbon_footprint = model.kwh(data)
print(f"Estimated monthly kwh: {carbon_footprint:.2f} kwh")

Estimated monthly kwh: 0.97 kwh
